In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from ipywidgets import interact,fixed,interactive

In [3]:
def ucitavanje_slika(putanja1,putanja2):
    
    global ref_slika,orig_slika,ref_r,ref_g,ref_b,orig_r,orig_g,orig_b
    # Učitavam slike na kojima ću vršiti obradu
    # Referentna slika je slika s koje ćemo uzimati uzorke palete boja
    # Originalna slika je slika koju ćemo menjati

    ref_slika = cv2.imread(putanja1)
    orig_slika = cv2.imread(putanja2)

    # Konvertovanje iz BGR u RGB

    ref_slika = cv2.cvtColor(ref_slika,cv2.COLOR_BGR2RGB)
    orig_slika = cv2.cvtColor(orig_slika,cv2.COLOR_BGR2RGB)


    # Izdvajanje crvene/zelene/plave boje referentne slike u odvojenu matricu

    ref_r = ref_slika[:,:,0] # ref_slika je trodimenzionalna matrica, te u ref_r kopiramo za svaki piksel vrednost RED kanala iz ref slike(ref_r je sada dvodim. matrica)
    ref_g = ref_slika[:,:,1]
    ref_b = ref_slika[:,:,2]


    # Izdvajanje crvene/zelene/plave boje originalne slike u odvojenu matricu

    orig_r = orig_slika[:,:,0]
    orig_g = orig_slika[:,:,1]
    orig_b = orig_slika[:,:,2]


    # Prikazujemo rezultate deljenja na kanale

    fig, ax = plt.subplots(2,4,figsize=(32,10))

    ax[0,0].imshow(ref_slika)
    ax[0,0].axis('off')
    ax[0,1].imshow(ref_r,cmap='jet')
    ax[0,1].axis('off')
    ax[0,2].imshow(ref_g,cmap='jet')
    ax[0,2].axis('off')
    ax[0,3].imshow(ref_b,cmap='jet')
    ax[0,3].axis('off')

    ax[1,0].imshow(orig_slika)
    ax[1,0].axis('off')
    ax[1,1].imshow(orig_r,cmap='jet')
    ax[1,1].axis('off')
    ax[1,2].imshow(orig_g,cmap='jet')
    ax[1,2].axis('off')
    ax[1,3].imshow(orig_b,cmap='jet')
    ax[1,3].axis('off')

    ax[0,0].set_title('Slika u boji (RGB)')
    ax[0,1].set_title('Slika u jednoj boji RED')
    ax[0,2].set_title('Slika u jednoj boji GREEN')
    ax[0,3].set_title('Slika u jednoj boji BLUE');

In [4]:
def histogrami():
    
    # kreiram 3 prazna niza duzine 256 za pamcenje normalizovanog histograma za 3 kanala referentne slike
    global hist_ref_r,hist_ref_g,hist_ref_b,hist_orig_r,hist_orig_b,hist_orig_g,origrows,origcols,refrows,refcols
    hist_ref_r = np.zeros(256)
    hist_ref_g = np.zeros(256)
    hist_ref_b = np.zeros(256)

    # kreiram i 3 prazna niza duzine 256 za pamćenje normalizovanog histograma za 3 kanala originalne slike

    hist_orig_r = np.zeros(256)
    hist_orig_g = np.zeros(256)
    hist_orig_b = np.zeros(256)

    # uzimam dimenzije referentne slike - broj piksela po širini i dužini

    refrows = ref_slika.shape[0]
    refcols = ref_slika.shape[1]

    # uzimam i dimenzije originalne slike

    origrows = orig_slika.shape[0]
    origcols = orig_slika.shape[1]


    # for petlja prolazi kroz svaki piksel po redovima i kolonama referentne slike

    for i in range(refrows):
        for j in range(refcols):
            hist_ref_r[ref_r[i][j]]+=1  # vrednost piksela u datom redu i koloni koristimo kao indeks
                                        # člana niza (histograma) na kojem povećavamo vrednost za 1 (broj pojavljivanja te vred.)
                                        # za svaku vrednost piksela iz matrice povećavamo broj ponavljanja te vrednosti na histogramu (u nizu na odgovarajućem indeksu)

    for i in range(refrows):   # ponavljamo postupak za kanal zelene boje
        for j in range(refcols):
            hist_ref_g[ref_g[i][j]]+=1   
                                        
    for i in range(refrows):   # ponavljamo postupak za kanal zelene boje
        for j in range(refcols):
            hist_ref_b[ref_b[i][j]]+=1
                                        

    # ponavljamo postupak za originalnu sliku

    for i in range(origrows):
        for j in range(origcols):
            hist_orig_r[orig_r[i][j]]+=1 # vrednost piksela u datom redu i koloni koristimo kao indeks 
                                        # člana niza (histograma) na kojem povećavamo vrednost za 1 (broj pojavljivanja te vred.)
                                        # za svaku vrednost piksela iz matrice povećavamo broj ponavljanja te vrednosti na histogramu (u nizu na odgovarajućem indeksu)

    for i in range(origrows):   # ponavljamo postupak za kanal zelene boje
        for j in range(origcols):
            hist_orig_g[orig_g[i][j]]+=1   
                                        
    for i in range(origrows):   # ponavljamo postupak za kanal zelene boje
        for j in range(origcols):
            hist_orig_b[orig_b[i][j]]+=1


In [5]:
def kumulativni_histogrami():
    
    global cdf_ref_r,cdf_ref_g,cdf_ref_b,cdf_orig_r,cdf_orig_g,cdf_orig_b
    # kreiram 3 prazna niza duzine 256 za pamcenje normalizovanog histograma za 3 kanala originalne slike

    cdf_ref_r = np.zeros(256)
    cdf_ref_g = np.zeros(256)
    cdf_ref_b = np.zeros(256)

    # ponavljam postupak za originalnu sliku

    cdf_orig_r = np.zeros(256)
    cdf_orig_g = np.zeros(256)
    cdf_orig_b = np.zeros(256)


    # for petlja koja prolazi kroz cele dužine nizova histogram za sva 3 kanala referentne slike

    cdf_ref_r[0]=hist_ref_r[0]
    for i in range(1,256):
        cdf_ref_r[i]=cdf_ref_r[i-1]+hist_ref_r[i]

    cdf_ref_g[0]=hist_ref_g[0]
    for i in range(1,256):
        cdf_ref_g[i]=cdf_ref_g[i-1]+hist_ref_g[i]

    cdf_ref_b[0]=hist_ref_b[0]
    for i in range(1,256):
        cdf_ref_b[i]=cdf_ref_b[i-1]+hist_ref_b[i]

    # ponavljamo postupak za originalnu sliku

    cdf_orig_r[0]=hist_orig_r[0]
    for i in range(1,256):
        cdf_orig_r[i]=cdf_orig_r[i-1]+hist_orig_r[i]

    cdf_orig_g[0]=hist_orig_g[0]
    for i in range(1,256):
        cdf_orig_g[i]=cdf_orig_g[i-1]+hist_orig_g[i]

    cdf_orig_b[0]=hist_orig_b[0]
    for i in range(1,256):
        cdf_orig_b[i]=cdf_orig_b[i-1]+hist_orig_b[i]

In [6]:
def prikaz_hist():

    global bins
    # PRIKAZ

    # pravim niz duzine 256 s vrednostima od 0 do 255, za granice binova koje nam trebaju za grupisanje vrednosti pri prikazu

    bins = np.arange(256)

    # plotovanje

    fig,ax1= plt.subplots(nrows=3,ncols=2,figsize=(15,15))

    ax2 = np.array([[ax.twinx() for ax in red] for red in ax1])

    ax1[0,0].bar(bins,hist_ref_r,color='b')
    ax1[1,0].bar(bins,hist_ref_g,color='b')
    ax1[2,0].bar(bins,hist_ref_b,color='b')
    ax1[0,1].bar(bins,hist_orig_r,color='b')
    ax1[1,1].bar(bins,hist_orig_g,color='b')
    ax1[2,1].bar(bins,hist_orig_b,color='b')
    ax2[0,0].plot(bins, cdf_ref_r, color='r')
    ax2[1,0].plot(bins, cdf_ref_g, color='r')
    ax2[2,0].plot(bins, cdf_ref_b, color='r')
    ax2[0,1].plot(bins, cdf_orig_r, color='r')
    ax2[1,1].plot(bins, cdf_orig_g, color='r')
    ax2[2,1].plot(bins, cdf_orig_b, color='r')

    ax1[0,0].set_title('Referentna slika')
    ax1[0,1].set_title('Originalna slika')
    ax1[0,0].set_ylabel('R')
    ax1[1,0].set_ylabel('G')
    ax1[2,0].set_ylabel('B')



In [7]:
def normalizacija():

    # Računamo dimenzije slika
    
    ref_dim = ref_slika.shape[0]*ref_slika.shape[1]

    orig_dim = orig_slika.shape[0]*orig_slika.shape[1]


    # for petlja prolazi kroz sve elemente nizova i svaki podeli s dimenzijama odgovarajuće slike

    for i in range(256):
        cdf_ref_r[i]=cdf_ref_r[i]/ref_dim

    for i in range(256):
        cdf_ref_g[i]=cdf_ref_g[i]/ref_dim

    for i in range(256):
        cdf_ref_b[i]=cdf_ref_b[i]/ref_dim

    # ponavljamo postupak za originalnu sliku

    for i in range(256):
        cdf_orig_r[i]=cdf_orig_r[i]/orig_dim

    for i in range(256):
        cdf_orig_g[i]=cdf_orig_g[i]/orig_dim

    for i in range(256):
        cdf_orig_b[i]=cdf_orig_b[i]/orig_dim


    fig, ax = plt.subplots(3,2,figsize=(15,10))

    ax[0,0].plot(cdf_ref_r,color='r')
    ax[1,0].plot(cdf_ref_g,color='r')
    ax[2,0].plot(cdf_ref_b,color='r')
    ax[0,1].plot(cdf_orig_r,color='r')
    ax[1,1].plot(cdf_orig_g,color='r')
    ax[2,1].plot(cdf_orig_b,color='r')

    ax[0,0].set_title('Referentna slika')
    ax[0,1].set_title('Originalna slika')
    ax[0,0].set_ylabel('R')
    ax[1,0].set_ylabel('G')
    ax[2,0].set_ylabel('B');

In [8]:
def specifikacija():
    global ciljna_slika
    M_r = []
    M_g = []
    M_b = []

    for i in range(256):
        t = cdf_ref_r[i]
        najblizi_index = np.abs(cdf_orig_r - t).argmin()
        M_r.append(najblizi_index)

    for i in range(256):
        t = cdf_ref_r[i]
        najblizi_index = np.abs(cdf_orig_g - t).argmin()
        M_g.append(najblizi_index)

    for i in range(256):
        t = cdf_ref_r[i]
        najblizi_index = np.abs(cdf_orig_b - t).argmin()
        M_b.append(najblizi_index)
    
    cilj_r = orig_r.copy()
    cilj_g = orig_g.copy()
    cilj_b = orig_b.copy()
    ciljna_slika=orig_slika.copy()

    for i in range(origrows):
        for j in range(origcols):
            cilj_r[i][j]=M_r[cilj_r[i][j]]


    for i in range(origrows):
        for j in range(origcols):
            cilj_g[i][j]=M_g[cilj_g[i][j]]

            
    for i in range(origrows):
        for j in range(origcols):
            cilj_b[i][j]=M_b[cilj_b[i][j]]

    cilj_r[215][141]
    for i in range(origrows):
        for j in range(origcols):
            ciljna_slika[i][j][0]=cilj_r[i][j]
            ciljna_slika[i][j][1]=cilj_g[i][j]
            ciljna_slika[i][j][2]=cilj_b[i][j]

In [9]:
def prikaz_rezultata():
    fig, ax = plt.subplots(1,3,figsize=(22,8))
    ax[0].imshow(ref_slika)
    ax[1].imshow(orig_slika)
    ax[2].imshow(ciljna_slika)
    ax[0].axis('off')
    ax[1].axis('off')
    ax[2].axis('off')
    ax[0].set_title('Referentna slika')
    ax[1].set_title('Originalna slika')
    ax[2].set_title('Izmenjena slika');

In [17]:
putanja1='Slike/dolina.png'
putanja2='Slike/palme.png'
interactive(ucitavanje_slika, putanja1=fixed(putanja1),putanja2=fixed(putanja2))

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [18]:
interactive(histogrami)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [19]:
interactive(kumulativni_histogrami)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [20]:
interactive(prikaz_hist)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [21]:
interactive(normalizacija)

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [22]:
interactive(specifikacija)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [23]:
interactive(prikaz_rezultata)

interactive(children=(Output(),), _dom_classes=('widget-interact',))